# Project: **Image Denoising using Haar Wavelets**

### Author: *Mario Medone*

##### Course: *Digital Signal & Image Processing*

<br>

In this notebook, we will speak about **image denoising**, in particular about how the wavelet transform can be useful to enhance the quality of an image affected by noise.

After a brief introduction on the 2D Haar wavelet transform, we will speak about a method to denoise images through multiple levels wavelet decomposition which is based on a thresholding function: we will present different types of it, such as **hard thresholding** and **soft thresholding**. We will also speak about the best possible choice of the threshold parameter required by the thresholding function: firstly, we will introduce the empirical method to choose the best value for the parameter, then we will provide some methods to compute it, such as the **VisuShrink**, **BayesShrink** and a modified version of the first one, which we called **VisuShrinkMod**.

To do this, we will see some experiments using the *Lena* image and we will comment the results.
<br><br>

---

## 1. Introduction

Nowadays, cameras and images have become common parts of our life. Anyone with a device equipped with a camera can take a photograph or video, then edit, save and share it. Not only this. Cameras can be useful for various activities: for example, intelligent traffic system, bio-medical imaging, image processing for medical diagnostic, multimedia application, object tracking, and many more.

So, there is a great interest in this technological field, and there are many many studies that deepen every facet.

<br>

The one we want to focus on is that the images are most often damaged by noise. Noise can be defined as any unwanted interference in image data and, typically, it corresponds to higher frequencies. Noise can appear in images from multiple sources, for example during the acquisition step or the transmission process.

As a result, denoising is one of the major challenges in the field of image processing to improve image quality: this is the process that permits to estimate the original image by suppressing the noise in a noisy image.

In mathematical notation, a noisy image can be described as in the equation below

$$ Img_N(x,y) = Img(x,y) + N(x,y) $$

where $ Img(x,y) $ is the original image, $ N(x,y) $ is the noise, and $ Img_N(x,y) $ is the resulting noisy image.

<br>Noises usually present in an image are uniform noise, noise impulse, salt & pepper, Gaussian noise, or others. In this work, we focus on Gaussian noise: this type of noise can include other types of noise implicitly and it is the most frequent noise that appears in the images. We assume that it has a probability density function equal to the one of the normal distribution, also called Gaussian distribution. The probability density function $ p $ of a Gaussian random variable $ z $ is given by

$$ p(z) = \frac{1}{\sqrt{2\pi\sigma^2}} \cdot e^{-\frac{(z-\mu)^2}{2\sigma^2}} $$

where $ \mu $ is the mean value and $ \sigma $ is the standard deviation.

<br>Furthermore, the Gaussian noise has the property that, after a 2D wavelet transform, it remains independent from each other at different frequency bands. If we increase the level decomposition of the wavelet transform, the noise energy reduces rapidly, instead the original image signal keeps its local maxima in the same location and its energy will not reduce rapidly as the noise one. So, this is a very important feature that can permit to obtain good image denoising results, by thresholding the wavelet coefficients in the right way.

<br>This notebook is organized as follows:

- section 2 is devoted for literature review which includes a brief introduction to 2D Haar wavelet transform, the denoising procedure and the wavelet thresholding methods; there is also a long discussion on how to choose the optimal value for the threshold parameter;

- in section 3, we implement the introduced thresholding methods and the denoising procedure;

- In section 4, we test what we have implemented to make some conclusion about which thresholding method (and parameter) can be the optimal one;

- finally, in section 5, we write our conclusions and considerations about all the work.
<br><br>

---

In [ ]:
# Import required modules
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pywt


## 2. Literature review

We give a brief introduction about what is the 2D Haar wavelet transform and how it works. Furthermore, we speak a little about the denoising method we will implement, the thresholding methods and about how we can choose the best value for the threshold parameter.

### 2.1. 2D Haar wavelet transform

Starting from what we have learned in class about the 1D Haar wavelets, we know that a one dimensional image can be treated as sequences of coefficients, approximation and detail ones.
<br>We also introduced the *scaling function* $ \phi $ and the *wavelet function* $ \psi $ that are useful in the wavelet decomposition process.

<br>Thus, the wavelet transform for an image, as a 2D signal, will be obtained from 1D wavelet tranform. We can get the scaling function and wavelet function for 2D by multiplying two 1D functions. The scaling function is obtained by multipling two 1D scaling ones: $ \phi(x,y) = \phi(x)\phi(y) $. The wavelet functions are obtained by multiplying two wavelet functions or one wavelet and one scaling function for 1D case. In the 2D case, there exist three wavelet functions that scan details in:
- horizontal, $ \psi^{(1)}(x,y) = \phi(x)\cdot\psi(y) $;
- vertical, $ \psi^{(2)}(x,y) = \psi(x)\cdot\phi(y) $;
- diagonal, $ \psi^{(3)}(x,y) = \psi(x)\cdot\psi(y) $.

This may be represented as a four channels perfect decomposition filter bank as shown below.
<br><br>
<center width="100%" style="padding:10px"><img src="../docs/images/image1.png" width="650px"></center>

<br>In particular for a 2D Haar wavelet transform of an image, $ a_L $ represents a low pass filter and $ a_H $ represents a high pass filter. First, each row of the image is filtered by $ a_L $ and $ a_H $, and then the filtered results are down-sampled by 2. After that, each of the resulting columns is further low pass and high pass filtered using $ a_L $ and $ a_H $. Again we down-sampling by 2, and we finally obtain the images decomposed into four different sub-bands. There exist three types of detail coefficients for each resolution: horizontal detail ($ HL $), vertical detail ($ LH $) and diagonal one ($ HH $). $ LL $ corresponds to the approximation of the image, on which we can repeat the operations using the second stage of identical filter bank (like before). Thus, a typical 2D wavelet transform can generate hierarchical structure as the one shown below.
<br><br>
<center width="100%" style="padding:10px"><img src="../docs/images/image2.png" width="650px"></center>

<br>Starting from a 2D Haar wavelets structure, we can reconstruct the original image, processing it through the *2D inverse Haar wavelet transform*.

<br>For our purpose, we will use an already implemented version of the 2D Haar wavelet transform, focusing more our attention on the image denoising part.


### 2.2. Denoising procedure

Up to this point, we have understood why wavelet transform is a good utility to clean images from noise and how wavelets work.

Now, we want to unerdstand the approach to do image denoising using wavelet transform. The work we have to do can be summarized in the following simple steps:

1. Apply multiple levels 2D Haar wavelet transform on a noisy image; the image will be divided into four sub-bands: the low-low ($ LL_k $) that will contain the approximation coefficients and the other three high-frequency sub-bands ($ HL_k $, $ LH_k $ and $ HH_k $) that will contain the different detail coefficients (horizontal, vertical and diagonal), with $k$ the level of decomposition.

 After the first level, the $ k $ level 2D Haar wavelet transform will be applied only to the $ LL_{k-1} $ to obtain the subsequent decomposition in $ LL_k $, $ HL_k $, $ LH_k $ and $ HH_k $ sub-bands;

2. Apply a thresholding method on the three high-frequency sub-bands; we have to define a threshold value and pass it, together with each sub-band, to the thresholding method which operates over each pixel of the sub-band as follows:
 - if the pixel value is less than the threshold value, it sets the pixel value to zero;
 - otherwise, leave it, or modify it (a little), and skip to the next pixel;
 
 This step is repeated until all the pixels have been visited;

3. Apply one level 2D inverse Haar wavelet transform on the latest four sub-bands ($ LL_k $, $ HL_k $, $ LH_k $ and $ HH_k $) to obtain a denoised $ LL_{k-1} $ sub-band, possibly without noise;

4. Step 2 and 3 are repeated at each level of decomposition, until we rebuild the (possibly) denoised image with the latest inverse wavelet transform (using $ LL_1 $, $ HL_1 $, $ LH_1 $ and $ HH_1 $ sub-bands to obtain the final image).

4. Finally, we can use different types of metrics (such as *MSE* or *PSNR*) to compare the new image and the original one and, so, to evalute the denoising method.

<br>As we have said before, noise mainly dominates the detail coefficients in the wavelet transform; if we can set a reasonable threshold, we can drop all the noise contaminated detail coefficients to zero to remove noise from our images. The following question is how we use the threshold to reduce
noise in an image and the crucial ingredient in this procedure is: how to decide the value of the threshold and the thresholding function? There are many possible solutions, we will explain and compare some methods in detail.


### 2.3. How do we do thresholding?

The key problem to the image denoising of wavelet is how to carry on correction to the coefficients of transformation.

Let $ W $ be a wavelet decomposition detail sub-band (horizontal, vertical or diagonal). Let us apply threshold zeroing with a fixed threshold value $ T $ for $ W $. Let us denote the thresholded sub-band as $ W' $.
<br>Having said that, we can introduce two ways to perform thresholding: **hard thresholding** and **soft thresholding**.


#### 2.3.1. Hard Thresholding

The hard thresholding strategy compares the absolute value of the wavelet coefficients with a threshold value of $ T $. It preserves the wavelet coefficients whose absolute values are greater than the threshold. If the wavelet coefficients are below the threshold, they are setted to zero.

It can be described with the following equation:

$$ W'(x,y) = \begin{cases} W(x,y), & |W(x,y)| \ge T \\ 0, & |W(x,y)| < T \end{cases} $$

<br>

As shown in the representation below, the hard thresholding function is discontinuous. Although it has the advantage of retaining the image local features (such as edges), it may produce visual distortion in the reconstructed image.
<br><br>
<center width="100%" style="padding:10px"><img src="../docs/images/image3.png" width="500px"></center>


#### 2.3.2. Soft Thresholding

The soft thresholding function is very similar to hard thresholding but it shifts wavelet coefficients towards zero. The wavelet coefficients whose absolute values are greater than $ T $ are subtracted by $ T $, and all of the other coefficients are set to zero. After soft thresholding, the wavelet coefficients are smoother in the wavelet domain and, also, the reconstructed image looks smoother than the one obtained with the hard thresholding strategy.

It can be described with the following equation:

$$ W'(x,y) = \begin{cases} \text{sign}(W(x,y)) \cdot (|W(x,y)|-T), & |W(x,y)| \ge T \\ 0, & |W(x,y)| < T \end{cases} $$

<br>

We can see how the soft threshold works in the representation below.
<br><br>
<center width="100%" style="padding:10px"><img src="../docs/images/image4.png" width="500px"></center>

<br>

We can now implement these two thresholding methods in Python.


In [ ]:
# Function that applies the selected thresholding method to the sub-band S using threshold value T
def thresholding(S, thresh_method, T):
  
  # Check the threshold value: if it does not respect the rule, raise an error
  if T < 0:
    raise ValueError("The threshold value T must be greater than zero!")

  # Hard thresholding
  if thresh_method == "hard":
    S[np.abs(S) < T] = 0
    return S

  # Soft thesholding
  elif thresh_method == "soft":
    S_new = np.sign(S) * (np.abs(S) - T)
    S_new[np.abs(S) < T] = 0
    return S_new

  # If thresh_method is not equal to one of the before, it is incorrect: raise an error
  else:
    raise ValueError("The selected thresholding method does not exist!")
    

#### 2.3.3. The threshold value

As we have already said, the selection of the threshold is critical for the performance of denoising images using this method. If the value is bigger, it would also remove the useful part of the signal, such as important image features; instead, if the value is too small, it is possible that it cannot remove noise sufficiently or, at worst, it would not be able to clean the image from noise at all.

For this work, we mainly focus on two of the most commonly used methods to define the threshold: *VisuShrink* and *BayesShrink*, introducing also a modified version of the first which we called *VisuShrinkMod*.

But, before we have a look to their characteristics, we want to take a step back and observe how the results vary as the threshold value varies and, therefore, the impact it has on the denoising algorithm.



Suppose we have done a one level 2D Haar wavelet decomposition: we have the approximation coefficients ($ LL $ sub-band) and the three detail coefficients ($ LH $, $ HL $, $ HH $ sub-bands). We want to apply one of the thresholding methods we introduce. So, we pick each of the high-frequency sub-bands and we apply the thresholding method. But we have to pass also a threshold value. How do we decide the value of the threshold?

Helping us with a toy image, we will see through a one level wavelet decomposition how the threshold modifies the detail coefficients and, thus, the image itself.

<br>

At the beginning, we can load the image and test directly the thresholding methods on it, without add the noise.

In [ ]:
# Load toy image
url = "../data/images/toy1.png"
original = cv2.imread(url, cv2.IMREAD_GRAYSCALE)

# Show the image
plt.figure(figsize=(7, 7))
plt.imshow(original, cmap=plt.cm.gray)
plt.title("Toy - Original image", pad=10)

plt.show()

We also define a method which permits to plot the four sub-bands without having to rewrite all these lines of code every time.


In [ ]:
# Function that permits to plot the four sub-bands of a wavelet decomposition
def plotSubBands(LL, LH, HL, HH):
  
  plt.subplot(1, 4, 1)
  plt.imshow(LL, cmap=plt.cm.gray)
  plt.title("LL sub-band (approximation coefficients)", pad=10)

  plt.subplot(1, 4, 2)
  plt.imshow(LH, cmap=plt.cm.gray)
  plt.title("LH sub-band (horizontal detail coefficients)", pad=10)

  plt.subplot(1, 4, 3)
  plt.imshow(HL, cmap=plt.cm.gray)
  plt.title("HL sub-band (vertical detail coefficients)", pad=10)

  plt.subplot(1, 4, 4)
  plt.imshow(HH, cmap=plt.cm.gray)
  plt.title("HH sub-band (diagonal detail coefficients)", pad=10)

  plt.show()
  

In [ ]:
# Apply a one level 2D Haar wavelet decomposition to obtain the four sub-bands
LL, (LH, HL, HH) = pywt.dwt2(original, "haar")

# Show them
plt.figure(figsize=(28, 7))
plotSubBands(LL, LH, HL, HH)


Now we apply the *hard thresholding* method using three different values for the threshold parameter. We will plot the sub-bands in each of the cases and we will comment the results.


In [ ]:
# Apply the hard thresholding with three values for the threshold parameter and see what happens
threshold_values = [10, 50, 90]
thresholded_images = []

for threshold_value in threshold_values:

  LH_new = thresholding(S=LH.copy(), thresh_method="hard", T=threshold_value)
  HL_new = thresholding(S=HL.copy(), thresh_method="hard", T=threshold_value)
  HH_new = thresholding(S=HH.copy(), thresh_method="hard", T=threshold_value)

  # Show the sub-bands
  print("Threshold value: {}\n".format(threshold_value))
  plt.figure(figsize=(28, 7))
  plotSubBands(LL, LH_new, HL_new, HH_new)
  plt.show()

  # Rebuild the image using the thresholded detail sub-bands
  thresholded_image = pywt.idwt2((LL, (LH_new, HL_new, HH_new)), "haar")
  thresholded_image[thresholded_image < 0] = 0
  thresholded_image[thresholded_image > 255] = 255
  thresholded_images.append(np.round(thresholded_image))

  print("\n-------------------------------------------------------------------\n\n")

hard_representative = thresholded_images[1]

# Show the images before and after hard thresholding
plt.figure(figsize=(15, 15))

plt.subplot(2, 2, 1)
plt.imshow(original, cmap=plt.cm.gray)
plt.title("Original image", pad=10)

plt.subplot(2, 2, 2)
plt.imshow(thresholded_images[0], cmap=plt.cm.gray)
plt.title("Thresholded image - threshold = {}".format(threshold_values[0]), pad=10)

plt.subplot(2, 2, 3)
plt.imshow(thresholded_images[1], cmap=plt.cm.gray)
plt.title("Thresholded image - threshold = {}".format(threshold_values[1]), pad=10)

plt.subplot(2, 2, 4)
plt.imshow(thresholded_images[2], cmap=plt.cm.gray)
plt.title("Thresholded image - threshold = {}".format(threshold_values[2]), pad=10)

plt.show()


What we can see through the plot of the detail coefficients is that with a higher value of the threshold parameter, more of the coefficients are set to zero, as we expected. 

So, what does it happen in the image? When we will apply the 2D Haar inverse transform with the thresholded detail coefficients, we will obtain an image that appears more granulated, particularly on the edges of the objects in it. This is because, for example, two nearby pixels that has a similar gray level will be represented by an approximation coefficient (the mean of the two) and a detail coefficient that will be very small because the two gray levels are very similar. So, if we apply the thresholding scheme, this detail will be set to zero and, when we rebuild the image, these two pixels will have the same value.

With hard thresholding, if the detail value is greater than the threshold, nothing is done: this is why we obtain a more granulated result respect, for example, to soft thresholding.


Now we apply the *soft thresholding* method using three different values for the threshold parameter. We will plot the sub-bands in each of the cases and we will comment the results.


In [ ]:
# Apply the soft thresholding with three values for the threshold parameter and see what happens
threshold_values = [10, 50, 90]
thresholded_images = []

for threshold_value in threshold_values:
  
  LH_new = thresholding(S=LH.copy(), thresh_method="soft", T=threshold_value)
  HL_new = thresholding(S=HL.copy(), thresh_method="soft", T=threshold_value)
  HH_new = thresholding(S=HH.copy(), thresh_method="soft", T=threshold_value)

  # Show the sub-bands
  print("Threshold value: {}\n".format(threshold_value))
  plt.figure(figsize=(28, 7))
  plotSubBands(LL, LH_new, HL_new, HH_new)
  plt.show()

  # Rebuild the image using the thresholded detail sub-bands
  thresholded_image = pywt.idwt2((LL, (LH_new, HL_new, HH_new)), "haar")
  thresholded_image[thresholded_image < 0] = 0
  thresholded_image[thresholded_image > 255] = 255
  thresholded_images.append(np.round(thresholded_image))

  print("\n-------------------------------------------------------------------\n\n")

soft_representative = thresholded_images[1]

# Show the images before and after hard thresholding
plt.figure(figsize=(15, 15))

plt.subplot(2, 2, 1)
plt.imshow(original, cmap=plt.cm.gray)
plt.title("Original image", pad=10)

plt.subplot(2, 2, 2)
plt.imshow(thresholded_images[0], cmap=plt.cm.gray)
plt.title("Thresholded image - threshold = {}".format(threshold_values[0]), pad=10)

plt.subplot(2, 2, 3)
plt.imshow(thresholded_images[1], cmap=plt.cm.gray)
plt.title("Thresholded image - threshold = {}".format(threshold_values[1]), pad=10)

plt.subplot(2, 2, 4)
plt.imshow(thresholded_images[2], cmap=plt.cm.gray)
plt.title("Thresholded image - threshold = {}".format(threshold_values[2]), pad=10)

plt.show()


What we can see through the plot of the detail coefficients is that with a higher value of the threshold parameter, more of the coefficients are set to zero, as we expected, and the coefficient that are greater than the threhsold value are shifted by the threshold value itself, appearing more attenuated.

So, what does it happen in the image? When we will apply the 2D Haar inverse transform with the thresholded detail coefficients, we will obtain an image that appears less granulated but more smoother and blurrier than the one obtained with the hard thresholding. This is because, for example, two nearby pixels that has not a similar gray level will be represented by an approximation coefficient (the mean of the two) and a detail coefficient that will be a number (for example 100). So, if we apply the thresholding scheme with a threshold value of 80, this detail will be reduced a lot (to 20) and, when we rebuild the image, these two pixels will be very similar in term of gray level and we will obtain a smoother and bluerrier image than before.


So, now we can make a comparison between hard and soft thresholding, with a fixed threshold parameter.


In [ ]:
# The fixed threshold parameter used to obtain hard_representative and soft_representative is 50
plt.figure(figsize=(30, 10))

plt.subplot(1, 3, 1)
plt.imshow(hard_representative, cmap=plt.cm.gray)
plt.title("Hard thresholding - threshold = 50", pad=10)

plt.subplot(1, 3, 2)
plt.imshow(soft_representative, cmap=plt.cm.gray)
plt.title("Soft thresholding - threshold = 50", pad=10)

plt.subplot(1, 3, 3)
plt.imshow(np.abs(hard_representative-soft_representative), cmap=plt.cm.gray)
plt.title("Absolute difference: Hard vs. Soft", pad=10)

plt.show()


As we just said, with hard thresholding we obtain an image in which the edges of the objects are more granulated, instead, with the soft one, they are more blurried.

We also plot the difference between the two methods to emphasize even more that the most difference is in how the two methods treat the edges in the image.


#### 2.3.3.1. How do we start to do denoising? 
#### *The choice of the optimal threshold value*

We know how to deal with images through the 2D Haar wavelet transform and we also know the behavior of hard and soft thresholding. Now, the question is: how can we choose the best possible threshold value? Does it changes if we use multiple levels decomposition?

So, we will see how we can manually tune the threshold parameter and the best choice for it.

<br>

Thus, we firstly have to define a method that permits to add Gaussian noise to an image: we define *addingNoise()* funciton to reach this scope.


In [ ]:
# Function that add Gaussian noise to an image
def addingNoise(original, mean, std):
  
  # Set a seed for reproducibility purpose
  np.random.seed(0)
  # Compute gaussian noise with mean and std passed to the function (mean is typically 0)
  gaussian = np.random.normal(mean, std, original.shape)
  # We add the noise to the original image and adapt the noisy image to the range 0-255
  noisy = np.round(original + gaussian)
  noisy[noisy > 255] = 255
  noisy[noisy < 0] = 0

  # Return the noisy image in uint8 values
  return np.uint8(noisy)


Then, we import our test image (*Lena* image) and generate a noisy version of it, adding a Gaussian noise with standard deviation equals to 10.

In [ ]:
# Load the Lena image and plot it
url = "../data/images/lena.gif"
original = cv2.imread(url, cv2.IMREAD_GRAYSCALE)

# Noisy image
noisy = addingNoise(original, 0, 10)

# Show the images
plt.figure(figsize=(15, 7))

plt.subplot(1,2,1)
plt.imshow(original, cmap=plt.cm.gray)
plt.title("Lena - Original image", pad=10)

plt.subplot(1,2,2)
plt.imshow(noisy, cmap=plt.cm.gray)
plt.title("Lena - Noisy image", pad=10)

plt.show()


We consider only soft thresholding and we fix a set of possible threshold values to try.

We will use as quality metric the **MSE** (*Mean Square Error*) which returns the error we incur in denoising the image, using a specific method, respect to the original image. Lower values indicate better performances.
<br>It is defined as in the equation below:

$$ MSE = \frac{1}{\text{wd}\cdot\text{ht}} \cdot\sum_{x=1}^{\text{ht}}\sum_{y=1}^{\text{wd}}\big(Img_{N_{x,y}} - Img_{x,y}\big)^2 $$

where $ Img_{N_{x,y}} $ and $ Img_{x,y} $ denote respectively the noisy and the original image pixel value in the $ i^{th} $ row and $ j^{th} $ column, $ \text{wd} $ and $ \text{ht} $ represent the width and height of the image.

<br>

Firstly, we do the work only considering one level 2D Haar wavelet transform.


In [ ]:
# Function to compute the MSE
def MSE(modify, original):

  # Try to commute the data matrix in float64 values
  try:
    modify = modify.astype(np.float64)
    original = original.astype(np.float64)

  # If it can't, raise an error
  except:
    raise TypeError("Bad data passed to the method!")

  # Compute MSE and return it
  return np.mean((modify-original)**2)


In [ ]:
# Define some threshold values and the thresholding method
threshold_values = np.arange(10, 91, 20)
thresh_method = "soft"

plt.figure(figsize=(30, 20))

plt.subplot(2, 3, 1)
plt.imshow(noisy, cmap=plt.cm.gray)
mse = np.round(MSE(noisy, original), 4)
plt.title("Noisy image - Gaussian noise, sigma = 10 - MSE: {}".format(mse), pad=10)

iter = 2
for threshold_value in threshold_values:

  # Do a one level 2D Haar wavelet transform
  LL, (LH, HL, HH) = pywt.dwt2(noisy, "haar")

  # Apply the thresholding method to each of the high-frequency sub-bands given the threshold_value
  LH_ = thresholding(LH, thresh_method, threshold_value)
  HL_ = thresholding(HL, thresh_method, threshold_value)
  HH_ = thresholding(HH, thresh_method, threshold_value)

  # Reconstruct the image using the 2D Haar inverse wavelet transform
  LL = pywt.idwt2((LL, (LH_, HL_, HH_)), "haar")

  # Adapt the final (denoised) image to the range 0-255 with uint8 values
  LL[LL < 0] = 0
  LL[LL > 255] = 255
  denoised_image = np.uint8(LL)

  plt.subplot(2, 3, iter)
  plt.imshow(denoised_image, cmap=plt.cm.gray)
  mse = np.round(MSE(denoised_image, original), 4)
  plt.title("Denoised image - Soft thresholding - threshold = {} - MSE: {}".format(
    threshold_value, mse), pad=10)

  iter += 1

plt.show()


By the results obtained, we can see that one pass of the Haar wavelet is not sufficient to do a good denoising work on the image. We can only see through our quality parameter (MSE) that the optimal threshold value can be researched around 10 or 30.

So, we restrict the interval for the threshold value and we try to implement a three level Haar wavelet decomposition, applying at each level the same thresholding method and value.


In [ ]:
# Define some threshold values and the thresholding method
threshold_values = np.arange(5, 26, 5)
thresh_method = "soft"

plt.figure(figsize=(30, 20))

plt.subplot(2,3,1)
plt.imshow(noisy, cmap=plt.cm.gray)
mse = np.round(MSE(noisy, original), 4)
plt.title("Noisy image - Gaussian noise, sigma = 10 - MSE: {}".format(mse), pad=10)

iter = 2
for threshold_value in threshold_values:

  # Do a one level 2D Haar wavelet transform
  LL, (LH, HL, HH) = pywt.dwt2(noisy, "haar")

  # Do a second level 2D Haar wavelet transform
  LL2, (LH2, HL2, HH2) = pywt.dwt2(LL, "haar")

  # Do a third level 2D Haar wavelet transform
  LL3, (LH3, HL3, HH3) = pywt.dwt2(LL2, "haar")

  # Apply the thresholding method to each of the high-frequency sub-bands given the threshold value
  LH3_ = thresholding(LH3, thresh_method, threshold_value)
  HL3_ = thresholding(HL3, thresh_method, threshold_value)
  HH3_ = thresholding(HH3, thresh_method, threshold_value)

  # Reconstruct the LL sub-band using the 2D Haar inverse wavelet transform
  LL2 = pywt.idwt2((LL3, (LH3_, HL3_, HH3_)), "haar")

  # Apply the thresholding method to each of the high-frequency sub-bands given the threshold value
  LH2_ = thresholding(LH2, thresh_method, threshold_value)
  HL2_ = thresholding(HL2, thresh_method, threshold_value)
  HH2_ = thresholding(HH2, thresh_method, threshold_value)

  # Reconstruct the LL sub-band using the 2D Haar inverse wavelet transform
  LL = pywt.idwt2((LL2, (LH2_, HL2_, HH2_)), "haar")

  # Apply the thresholding method to each of the high-frequency sub-bands given the threshold value
  LH_ = thresholding(LH, thresh_method, threshold_value)
  HL_ = thresholding(HL, thresh_method, threshold_value)
  HH_ = thresholding(HH, thresh_method, threshold_value)

  # Reconstruct the image using the 2D Haar inverse wavelet transform
  LL = pywt.idwt2((LL, (LH_, HL_, HH_)), "haar")

  # Adapt the final (denoised) image to the range 0-255 with uint8 values
  LL[LL < 0] = 0
  LL[LL > 255] = 255
  denoised_image = np.uint8(LL)

  plt.subplot(2, 3, iter)
  plt.imshow(denoised_image, cmap=plt.cm.gray)
  mse = np.round(MSE(denoised_image, original), 4)
  plt.title("Denoised image - Soft thresholding - threshold = {} - MSE: {}".format(
    threshold_value, mse), pad=10)

  iter += 1

plt.show()


By increasing the number of levels of decomposition, we observe that as the threshold value increases, the image becomes increasingly grainy.

This happens because as we go down the level, we get an increasingly coarse approximation of the image and the relative detail coefficients are an integral part of the image itself rather than noise.

We can think in this way: the more we drop in level, the more our threshold must decrease since we will consider as noise only those pixels that are gradually more and more similar (therefore with detail coefficients close to zero) to their neighboring pixels.

After some tests, we decide to divide the threshold value at each level by the $ \text{level} $ factor.

So, we will implement the new denoising algorithm and we will comment the results.


In [ ]:
# Define some threshold values and the thresholding method
threshold_values = np.arange(7.5, 21.6, 3.5)
thresh_method = "soft"

plt.figure(figsize=(30, 20))

plt.subplot(2,3,1)
plt.imshow(noisy, cmap=plt.cm.gray)
mse = np.round(MSE(noisy, original), 4)
plt.title("Noisy image - Gaussian noise, sigma = 10 - MSE: {}".format(mse), pad=10)

iter = 2
for threshold_value in threshold_values:

  # Do a one level 2D Haar wavelet transform
  LL, (LH, HL, HH) = pywt.dwt2(noisy, "haar")

  # Do a second level 2D Haar wavelet transform
  LL2, (LH2, HL2, HH2) = pywt.dwt2(LL, "haar")

  # Do a third level 2D Haar wavelet transform
  LL3, (LH3, HL3, HH3) = pywt.dwt2(LL2, "haar")

  # Apply the thresholding method to each of the high-frequency sub-bands given the threshold value
  LH3_ = thresholding(LH3, thresh_method, threshold_value/3)
  HL3_ = thresholding(HL3, thresh_method, threshold_value/3)
  HH3_ = thresholding(HH3, thresh_method, threshold_value/3)

  # Reconstruct the LL sub-band using the 2D Haar inverse wavelet transform
  LL2 = pywt.idwt2((LL3, (LH3_, HL3_, HH3_)), "haar")

  # Apply the thresholding method to each of the high-frequency sub-bands given the threshold value
  LH2_ = thresholding(LH2, thresh_method, threshold_value/2)
  HL2_ = thresholding(HL2, thresh_method, threshold_value/2)
  HH2_ = thresholding(HH2, thresh_method, threshold_value/2)

  # Reconstruct the LL sub-band using the 2D Haar inverse wavelet transform
  LL = pywt.idwt2((LL2, (LH2_, HL2_, HH2_)), "haar")

  # Apply the thresholding method to each of the high-frequency sub-bands given the threshold value
  LH_ = thresholding(LH, thresh_method, threshold_value)
  HL_ = thresholding(HL, thresh_method, threshold_value)
  HH_ = thresholding(HH, thresh_method, threshold_value)

  # Reconstruct the image using the 2D Haar inverse wavelet transform
  LL = pywt.idwt2((LL, (LH_, HL_, HH_)), "haar")

  # Adapt the final (denoised) image to the range 0-255 with uint8 values
  LL[LL < 0] = 0
  LL[LL > 255] = 255
  denoised_image = np.uint8(LL)

  plt.subplot(2, 3, iter)
  plt.imshow(denoised_image, cmap=plt.cm.gray)
  mse = np.round(MSE(denoised_image, original), 4)
  plt.title("Denoised image - Soft thresholding - threshold = {} - MSE: {}".format(
    threshold_value, mse), pad=10)

  iter += 1

plt.show()


After this last step, we can see how we have just implemented a simple method that permits through three levels wavelet decomposition to clean an image from the noise, selecting the optimal threshold parameter using a quantitative value for the error (MSE) and a qualitative point of view (how the denoised image appears to our eyes, in terms of granularity, blur and smoothness).

In this case, we will choose $ 14.5 $ as value for the threshold parameter.

<br>

The last question is: does the threshold parameter change if the quantity of noise in the image increases?

We will try it with the same image adding a larger amount of noise ($ \sigma = 15 $) and the last version of our denoising procedure. We will test a larger interval for the threshold value and we will comment the results.


In [ ]:
# Noisy image
noisy = addingNoise(original, 0, 15)
# Define some threshold values and the thresholding method
threshold_values = np.arange(20, 91, 15)
thresh_method = "soft"

plt.figure(figsize=(30, 20))

plt.subplot(2,3,1)
plt.imshow(noisy, cmap=plt.cm.gray)
mse = np.round(MSE(noisy, original), 4)
plt.title("Noisy image - Gaussian noise, sigma = 15 - MSE: {}".format(mse), pad=10)

iter = 2
for threshold_value in threshold_values:

  # Do a one level 2D Haar wavelet transform
  LL, (LH, HL, HH) = pywt.dwt2(noisy, "haar")

  # Do a second level 2D Haar wavelet transform
  LL2, (LH2, HL2, HH2) = pywt.dwt2(LL, "haar")

  # Do a third level 2D Haar wavelet transform
  LL3, (LH3, HL3, HH3) = pywt.dwt2(LL2, "haar")

  # Apply the thresholding method to each of the high-frequency sub-bands given the threshold value
  LH3_ = thresholding(LH3, thresh_method, threshold_value/4)
  HL3_ = thresholding(HL3, thresh_method, threshold_value/4)
  HH3_ = thresholding(HH3, thresh_method, threshold_value/4)

  # Reconstruct the LL sub-band using the 2D Haar inverse wavelet transform
  LL2 = pywt.idwt2((LL3, (LH3_, HL3_, HH3_)), "haar")

  # Apply the thresholding method to each of the high-frequency sub-bands given the threshold value
  LH2_ = thresholding(LH2, thresh_method, threshold_value/3)
  HL2_ = thresholding(HL2, thresh_method, threshold_value/3)
  HH2_ = thresholding(HH2, thresh_method, threshold_value/3)

  # Reconstruct the LL sub-band using the 2D Haar inverse wavelet transform
  LL = pywt.idwt2((LL2, (LH2_, HL2_, HH2_)), "haar")

  # Apply the thresholding method to each of the high-frequency sub-bands given the threshold value
  LH_ = thresholding(LH, thresh_method, threshold_value/2)
  HL_ = thresholding(HL, thresh_method, threshold_value/2)
  HH_ = thresholding(HH, thresh_method, threshold_value/2)

  # Reconstruct the image using the 2D Haar inverse wavelet transform
  LL = pywt.idwt2((LL, (LH_, HL_, HH_)), "haar")

  # Adapt the final (denoised) image to the range 0-255 with uint8 values
  LL[LL < 0] = 0
  LL[LL > 255] = 255
  denoised_image = np.uint8(LL)

  plt.subplot(2, 3, iter)
  plt.imshow(denoised_image, cmap=plt.cm.gray)
  mse = np.round(MSE(denoised_image, original), 4)
  plt.title("Denoised image - Soft thresholding - threshold = {} - MSE: {}".format(
    threshold_value, mse), pad=10)

  iter += 1

plt.show()


What we obtain is exactly what we expected: with a higher amount of noise added to the image, the algorithm required a bigger value for the threshold parameter. Observing the results, an optimal value can be found near 50, which permits to achieve a good denoising performance on the image without affecting too much everything that is not noise, so the image signal.

The need to increase the value of this parameter is due to the fact that, now, the noise represents a clearer component that generates discrepancies between neighboring pixels that are also quite significant. These differences result in higher values of the detail coefficients and, therefore, this requires a higher threshold value.


Doing this job each time we want to clean an image from noise is crazy (or maybe not).

As we said before, there exists some methods that permits, with or without data information, to define an "optimal" value for the threshold parameter.

<br>

<b>*VisuShrink*</b> is thresholding by applying the *Universal threshold* porposed by Donoho and Johnstone. This threshold is given by:

$$ T = \sqrt{2 \cdot \hat\sigma^2 \cdot ln(N)} $$

where $ \hat\sigma^2 $ is the noise variance and $ N $ is the number of pixels in the image. 
<br>The noise variance is found by *Robust Median Estimator* and it can be seen as the median absolute deviation of the diagonal detail coefficients of the first level decomposition. It is given by:

$$ \hat\sigma^2 = \Bigg(\frac{\text{median}(|Y_{x,y}|)}{0.6745}\Bigg)^2, \quad Y_{x,y} \in HH_1 $$

<br>As we can understand, it is a *universal threshold* because it keeps the same value regardless of the level of decomposition at which it is.
<br>As we can see after, this is not the best possible value for the threshold parameter, in particular with high level of noise in the image, because it tends to smooth and blur too much.

<br>

<b>*BayesShrink*</b> can be seen as a method to find an adaptive threshold value that depends on the specific sub-band it is considering, in particular by its standard deviation. This threshold is given by:

$$ T = \frac{\hat\sigma^2}{\hat\sigma_S} $$

where:
- $ T $ is the threshold value;
- $ \hat\sigma^2 $ is the noise variance in the image (found with the *RME*, as before);
- $ \hat\sigma_S $ is the standard deviation for the specific high-frequency sub-band $ S $, given by: 

 $$ \hat\sigma_S = \sqrt{\text{max}(\hat\sigma_Y^2 - \hat\sigma^2, 0)} \text{ ,} \quad \text{where} \quad \hat\sigma_Y^2 = \frac{1}{N} \sum_{i,j = 1}^N Y_{i,j}^2 \quad \text{with} \quad Y_{i,j} \in S $$

<br>So, the threshold is inversely proportional to the standard deviation of $ S $, and proportional to the noise variance of the image. When $ \frac{\hat\sigma^2}{\hat\sigma_S} \ll 1 $, the signal is much stronger than the noise: the threshold value is chosen to be small in order to preserve most of the signal and remove some of the noise; instead, when $ \frac{\hat\sigma^2}{\hat\sigma_S} \gg 1 $, the noise dominates and the threshold is chosen to be large to remove the noise which has exacted the signal. Thus, this threshold choice adapts to both the signal and the noise characteristics as reflected
in the parameters $ \hat\sigma^2 $ and $ \hat\sigma_S $.

<br>

<b>*VisuShrinkMod*</b> is based on the *VisuShrink* threshold value, with the difference that now the value depends also on the decomposition level at which the algorithm is working. This can be defined as:

$$ T = \frac{\sqrt{2 \cdot \hat\sigma^2 \cdot ln(N)}}{l+1} $$

where $ l $ identifies the decomposition level.

If we think about what we said in the introduction, the noise energy rapidly decreases when the algorithm goes in deeper decomposition levels, instead the signal energy (of the image) remains stable. So, what we want to do is to decrease the value of threshold at each decomposition level to permits to not delete signal information but only the noisy one. This threshold implementation permits to do this.

<br>

So, we have three different type of threshold values:
- *VisuShrink*: depends only on the noise variance in the image and its size, it is always the same;

- *BayesShrink*: is an adaptive threshold that depends on the standard deviation of the specific sub-band, so it depends on the data it is analyzing;

- *VisuShrinkMod*: is very similar to the first one, but it depends on the decomposition level at which the algorithm is, regardless of the data available.

<br>

We just have to move on to the implementation phase of the methods to compute the threshold value and the denoising procedure, and to apply it on the test images.
<br><br>

---

## 3. Implementation part

In this section, we will implement the *optimal_T_value()* function, which permits to return the optimal value for the threshold respect to the selected type of threshold we want.

Then, we will implement the multiple levels 2D Haar wavelet transform denoising algorithm.


The *optimal_T_value()* function requires some parameters:
- $ \text{compute\_T\_method} $ : the method to use to define the value of threshold (*VisuShrink*, *BayesShrink* or *VisuShrinkMod*);
- $ \text{sigma\_hat} $ : the noise standard deviation calculated with the *RME* on the first level high-high sub-band ($ HH_1 $);
- $ \text{S} $ : the data matrix containing pixel values of a sub-band;
- $ \text{level} $ : decomposition level.

In [ ]:
# Function that returns the optimal threshold value respect to the choosen method
def optimal_T_value(compute_T_method, sigma_hat, S=[], level=0):

  # Check the noise standard deviation value: if it does not respect the rule, raise an error
  if sigma_hat < 0:
    raise ValueError("""The noise standard deviation value must be greater or equal 
                        than zero!""")

  # VisuShrink implementation, requires only the noise standard deviation parameter
  if compute_T_method == "VisuShrink":
      return sigma_hat * np.sqrt(2 * np.log(512**2))

  # BayesShrink implementation, requires the noise standard deviation and the image_data parameters
  elif compute_T_method == "BayesShrink":
      sigma_S = np.sqrt(np.max([np.mean(S**2) - sigma_hat**2, 1e-3]))
      return sigma_hat**2 / sigma_S

  # VisuShrinkMod implemetation, requires the noise standard deviation and the level parameters
  elif compute_T_method == "VisuShrinkMod":
    # Check the decomposition level value
    if level > 0:
      return optimal_T_value("VisuShrink", sigma_hat) / (level+1)
    # Raise an error if the level value is not correct
    else:
      raise ValueError("Level value needs to be greater than zero!")

  # If compute_T_method is not equal to one of the before, it is incorrect: raise an error
  else:
    raise ValueError("The selected method does not exist!")


Finally, we can implement the denoising procedure based on multiple level decomposition of the wavelets. We build a method that takes different parameters to guarantee some choices on its running:
- $ \text{image} $ : the data matrix containing pixel values of the image we want to clean from noise;
- $ \text{thresh\_method} $ : the method to use to do thresholding on the high-frequency sub-bands (*hard* or *soft*);
- $ \text{compute\_T\_method} $ : the method to use to define the value of threshold (*VisuShrink*, *BayesShrink* or *VisuShrinkMod*);
- $ \text{max\_level} $ : the maximum level of decomposition of the wavelets we want (-1 for the maximum possible level);
- $ \text{level} $ : **This parameter is only used by the method, the user has not to set it**.

In [ ]:
# Global variables
sigma_hat = -1
recursive = True

# Function that returns a denoised version of the given input image (noisy one)
def denoiseWavelet(image, thresh_method, compute_T_method, max_level=-1, level=1):

  # Recall global variables
  global sigma_hat, recursive
  
  # If this is the first level decomposition
  if level == 1:
    
    # If max_level is equal to -1, set it to the maximum possible number of decomposition levels of the wavelet
    if (max_level == -1):
      max_level = np.floor(np.log2(image.shape[0]))
    
    # If the passed value for max_level is incorrect, raise an error
    elif (max_level < -1) | (max_level == 0) | \
      (max_level > np.floor(np.log2(image.shape[0]))):
      raise ValueError("""Please, insert a correct max_level! 
                          Insert -1 if you want the maximum possible!""")

    # Set recursive to True to enter the while loop
    # It is necessary to recursively call this function until the max_level level of decomposition is reached
    recursive = True

  # Until max_level decomposition level is reached or recursive is True
  while (level <= max_level) & recursive:
    
    # Do a one level 2D Haar wavelet transform
    LL, (LH, HL, HH) = pywt.dwt2(image, "haar")

    # If this is the first level, update the value of sigma_hat (noise standard deviation of the first level high-high sub-band)
    if level == 1:
      sigma_hat = np.median(np.abs(HH))/0.6745

    # Increment the level value to the next one
    level += 1
    
    # Recursively call the denoiseWavelet method on the low-low sub-band to compute another level of decomposition
    LL, level = denoiseWavelet(LL, thresh_method, compute_T_method, max_level, level)
      
  # When exit the while loop, it means that it reaches the max_level level of decomposition
  # So, set recursive to False and start to go backward, thresholding the high-frequency sub-bands and 
  # rebuilding the image through each level of decomposition
  if recursive:
    recursive = False
    return image, level
  
  # Recursive is False: do the thresholding and rebuild the image of the current level
  else:
    
    # Decrease the level value of one
    level -= 1

    # Compute the threshold value for each of the high-frequency sub-bands
    T_LH = optimal_T_value(compute_T_method, sigma_hat, LH, level)
    T_HL = optimal_T_value(compute_T_method, sigma_hat, HL, level)
    T_HH = optimal_T_value(compute_T_method, sigma_hat, HH, level)

    # Apply the thresholding method to each of the high-frequency sub-bands given the threshold values just computed
    LH_ = thresholding(LH, thresh_method, T_LH)
    HL_ = thresholding(HL, thresh_method, T_HL)
    HH_ = thresholding(HH, thresh_method, T_HH)

    # Reconstruct the current level LL (low-low sub-band) using the 2D Haar inverse wavelet transform
    LL = pywt.idwt2((LL, (LH_, HL_, HH_)), "haar")

    # If this is the latest level of reconstruction of the image
    if level == 1:
      
      # Adapt the final (denoised) image to the range 0-255 with uint8 value
      LL[LL < 0] = 0
      LL[LL > 255] = 255
      LL = np.uint8(LL)
      
      # Return the denoised image
      return LL

    # If this is not the latest level, return the LL sub-band and the current level to close one recursive call of denoiseWavelet
    return LL, level


<br>

---

## 4. Experiments and results

The denoising threshold algorithms compared are *VisuShrink*, *BayesShrink* and *VisuShrinkMod*. 

In our implementation, *hard thresholding* and *soft thresholding* are used to evaluate these threshold selection algorithms. 

The wavelet used in our experiment is the 2D Haar discrete wavelet transform. 

We test the denoising procedure we implemented: it analyzes multiple level decomposition of the wavelets to better clean the image from the noise.

The experiments are conducted on a gray scale test image, *Lena*, of size 512 × 512 pixels at different noise levels: $\sigma \in \{5, \text{ } 7.5, \text{ } 10, \text{ } 12.5, \text{ } 15\} $.


### 4.1. Objective evaluation method

The performance of the proposed image denoising strategy in this work is evaluated in terms of PSNR (*Peak Signal-to-Noise Ratio*), for 8bit gray scale images. This is an engineering term for the ratio between the maximum possible power of a signal and the power of corrupting noise that affects the fidelity of its representation.

<br>In order to evaluate the PSNR value, we have to compute the MSE of an image. We can do this using the already implemented version of the MSE.

<br>Then, the objective quality of the reconstructed image is measured by the PSNR, computed as follows:

$$ PSNR = 10 \cdot log_{10}\Bigg(\frac{255^2}{MSE}\Bigg) dB $$

<br>

Then, now, we will implement this evaluation method.


In [ ]:
# Function that is used to evaluate the quality of denoised images with respect to the original one
def PSNR(modify, original):

  # Try to commute the data matrix in float64 values
  try:
    modify = modify.astype(np.float64)
    original = original.astype(np.float64)
  # If it can't, raise an error
  except:
    raise TypeError("Bad data passed to the method!")

  # Compute the MSE
  MSE_value = MSE(modify, original)
  # Compute the PSNR
  PSNR = 10*np.log10(255**2/MSE_value)
  
  # Return the PSNR
  return PSNR


### 4.2. Lena image test

We start reloading the *Lena* image.

Then, we will iterate over different list to test all the possible combinations between different $ \sigma $ values for noise, thresholding methods and algorithms to compute the optimal threshold value, using five decomposition levels of wavelets.

Once the results are obtained, we will rearrange them in a table to make them as explicit as possible and we will show some examples of bad/good denoised images.


In [ ]:
# Load the Lena image
url = "../data/images/lena.gif"
original = cv2.imread(url, cv2.IMREAD_GRAYSCALE)


In [ ]:
# Prepare the test list for each of the parameters
sigmas = np.arange(5, 15.1, 2.5)
T_methods = ["VisuShrink", "BayesShrink", "VisuShrinkMod"]
thresh_methods = ["hard", "soft"]

# Initialize the list which will contain the denoised images and the respective PSNR values
PSNRs_noisy = []
denoised_images = []
PSNRs_denoised = []

# Do all the test and fill the denoised_images and PSNRs lists
# For each sigma (noise)
for sigma in sigmas:
  # Create a noisy version of the image adding Gaussian noise with mean equals zero and standard deviation equal to sigma
  noisy = addingNoise(original, 0, sigma)
  PSNRs_noisy.append(PSNR(noisy, original))
  # For each of the methods to define the threshold value
  for T_method in T_methods:
    # For each of the thresholding methods
    for thresh_method in thresh_methods:
      # Apply the denoising method to the image and save the result in the denoised_images list
      denoised_images.append(denoiseWavelet(image=noisy, thresh_method=thresh_method, compute_T_method=T_method, max_level=5))
      # Compute the PSNR of the current denoised image with respect to the original one
      PSNRs_denoised.append(PSNR(denoised_images[-1], original))


In [ ]:
# Prepare multi index for rows
iterables = [sigmas, T_methods]
multi_idx = pd.MultiIndex.from_product(iterables, names=["sigma", "T_method"])
# Prepare index for columns
variables = pd.Index(thresh_methods, name="thresh_method")

# Reshape the data and build the dataframe
PSNRs_denoised = np.reshape(np.array(PSNRs_denoised), (15, 2))
PSNRs_denoised = pd.DataFrame(PSNRs_denoised, index=multi_idx, columns=variables)

# Rearrange the PSNR values for the noisy images
PSNRs_noisy = np.reshape(np.array(PSNRs_noisy), (1, 5))
PSNRs_noisy = pd.DataFrame(np.reshape(np.array(PSNRs_noisy), (1, 5)), 
                           columns=pd.Index(sigmas, name="sigma"))

# Show tables
print("PSNRs noisy images:")
display(PSNRs_noisy)
print("\n\n-----------------------------------------------------------------\n\n")
print("PSNRs denoised images:")
display(PSNRs_denoised)


As we can see from the tables above, in general, the PSNR value decreases when the amount of noise added to the image ($ \sigma $ value) increases: so, bigger is the noise, worst the denoising of the image is.

The first thing that immediately catches the eye is that the *VisuShrink* method to find a value for the threshold always performs worst respect to the other two. This can be justified by the fact that it is a fixed value for each level of decomposition and this does not fit with what has been said previously: at each deeper level, the noise energy decreases and, therefore, the threshold level must decrease with it.
What we expect are more and more distorted images as the noise increases.

We take as examples the *VisuShrink* with the two thresholding methods with three different levels of noise to highlight this behavior.


In [ ]:
plt.figure(figsize=(30, 20))

plt.subplot(2, 3, 1)
plt.imshow(denoised_images[0], cmap=plt.cm.gray)
plt.title("VisuShrink - Hard thresholding - sigma: 5 - PSNR: {}".format(
    np.round(PSNR(denoised_images[0], original), 4)))

plt.subplot(2, 3, 2)
plt.imshow(denoised_images[12], cmap=plt.cm.gray)
plt.title("VisuShrink - Hard thresholding - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(denoised_images[12], original), 4)))

plt.subplot(2, 3, 3)
plt.imshow(denoised_images[24], cmap=plt.cm.gray)
plt.title("VisuShrink - Hard thresholding - sigma: 15 - PSNR: {}".format(
    np.round(PSNR(denoised_images[24], original), 4)))

plt.subplot(2, 3, 4)
plt.imshow(denoised_images[1], cmap=plt.cm.gray)
plt.title("VisuShrink - Soft thresholding - sigma: 5 - PSNR: {}".format(
    np.round(PSNR(denoised_images[1], original), 4)))

plt.subplot(2, 3, 5)
plt.imshow(denoised_images[13], cmap=plt.cm.gray)
plt.title("VisuShrink - Soft thresholding - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(denoised_images[13], original), 4)))

plt.subplot(2, 3, 6)
plt.imshow(denoised_images[25], cmap=plt.cm.gray)
plt.title("VisuShrink - Soft thresholding - sigma: 15 - PSNR: {}".format(
    np.round(PSNR(denoised_images[25], original), 4)))

plt.show()


The results reported confirms what we have just said: particularly, this behavior is amplified with the soft thresholding because this method, as we studied, returns images with smoother and blurrier edges respect than the other thresholding method.

<br>

Going forward with the analysis, we can also see, from the table, that the hard thresholding method performs worse than the soft one several times. So, this technique seems to not be the best possible one to use, since it performs a very strong thresholding on the pixels values. Doing this, if we think about high value of noise, it is not able to differentiate real values and the ones that are affected by noise, then it maintains a certain amount of noise which leads to poor denoising performances.
<br>This respect exactly what we have learned in *The threshold value* section of this project.

To be more explicit on the results, we show what happens with hard thresholding methods using *BayesShrink* and *VisuShrinkMod* with $ \sigma \in \{5, 10, 15\} $.


In [ ]:
plt.figure(figsize=(30, 20))

plt.subplot(2, 3, 1)
plt.imshow(denoised_images[2], cmap=plt.cm.gray)
plt.title("BayesShrink - Hard thresholding - sigma: 5 - PSNR: {}".format(
    np.round(PSNR(denoised_images[2], original), 4)))

plt.subplot(2, 3, 2)
plt.imshow(denoised_images[14], cmap=plt.cm.gray)
plt.title("BayesShrink - Hard thresholding - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(denoised_images[14], original), 4)))

plt.subplot(2, 3, 3)
plt.imshow(denoised_images[26], cmap=plt.cm.gray)
plt.title("BayesShrink - Hard thresholding - sigma: 15 - PSNR: {}".format(
    np.round(PSNR(denoised_images[26], original), 4)))

plt.subplot(2, 3, 4)
plt.imshow(denoised_images[4], cmap=plt.cm.gray)
plt.title("VisuShrinkMod - Hard thresholding - sigma: 5 - PSNR: {}".format(
    np.round(PSNR(denoised_images[4], original), 4)))

plt.subplot(2, 3, 5)
plt.imshow(denoised_images[16], cmap=plt.cm.gray)
plt.title("VisuShrinkMod - Hard thresholding - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(denoised_images[16], original), 4)))

plt.subplot(2, 3, 6)
plt.imshow(denoised_images[28], cmap=plt.cm.gray)
plt.title("VisuShrinkMod - Hard thresholding - sigma: 15 - PSNR: {}".format(
    np.round(PSNR(denoised_images[28], original), 4)))

plt.show()


By comparing all the possible results obtained with hard thresholding and the methods to compute threshold, we can say that the best performances can be reached using the *VisuShrinkMod* method to compute the threshold parameter. It returns a denoised image which still contains a slight amount of noise (lower respect the other two) but it permits to preserve the edges of the objects in the image.

We report the final example for hard thresholding, to emphasize the best performance of the *VisuShrinkMod* method with this type of thresholding.


In [ ]:
plt.figure(figsize=(30, 10))

plt.subplot(1, 3, 1)
plt.imshow(original, cmap=plt.cm.gray)
plt.title("Original image", pad=10)

plt.subplot(1, 3, 2)
noisy_t = addingNoise(original, 0, 10)
plt.imshow(noisy_t, cmap=plt.cm.gray)
plt.title("Noisy image - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(noisy_t, original), 4)))

plt.subplot(1, 3, 3)
plt.imshow(denoised_images[16], cmap=plt.cm.gray)
plt.title("VisuShrinkMod - Hard thresholding - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(denoised_images[16], original), 4)))

plt.show()


Now, we look now at the *soft thresholding* method and its behavior depending on the method chosen to calculate the threshold, *VisuShrink* excluded.


In [ ]:
plt.figure(figsize=(30, 20))

plt.subplot(2, 3, 1)
plt.imshow(denoised_images[3], cmap=plt.cm.gray)
plt.title("BayesShrink - Soft thresholding - sigma: 5 - PSNR: {}".format(
    np.round(PSNR(denoised_images[3], original), 4)))

plt.subplot(2, 3, 2)
plt.imshow(denoised_images[15], cmap=plt.cm.gray)
plt.title("BayesShrink - Soft thresholding - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(denoised_images[15], original), 4)))

plt.subplot(2, 3, 3)
plt.imshow(denoised_images[27], cmap=plt.cm.gray)
plt.title("BayesShrink - Soft thresholding - sigma: 15 - PSNR: {}".format(
    np.round(PSNR(denoised_images[27], original), 4)))

plt.subplot(2, 3, 4)
plt.imshow(denoised_images[5], cmap=plt.cm.gray)
plt.title("VisuShrinkMod - Soft thresholding - sigma: 5 - PSNR: {}".format(
    np.round(PSNR(denoised_images[5], original), 4)))

plt.subplot(2, 3, 5)
plt.imshow(denoised_images[17], cmap=plt.cm.gray)
plt.title("VisuShrinkMod - Soft thresholding - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(denoised_images[17], original), 4)))

plt.subplot(2, 3, 6)
plt.imshow(denoised_images[29], cmap=plt.cm.gray)
plt.title("VisuShrinkMod - Soft thresholding - sigma: 15 - PSNR: {}".format(
    np.round(PSNR(denoised_images[29], original), 4)))

plt.show()


Looking at the soft thresholding results, we can say that the denoised images obtained through the *VisuShrinkMod* appear a little blurrier than the ones obtained using the *BayesShrink* method, also they are a little bit grainy.
<br>Using also the PSNR evaluation metric, we can confirm that the best results using soft thresholding method can be reached applying the *BayesShrink* method to define the threshold value during the denoising procedure.

Therefore, as before, we report a final example for soft thresholding, to emphasize the best performance of the BayesShrink method with this type of thresholding.


In [ ]:
plt.figure(figsize=(30, 10))

plt.subplot(1, 3, 1)
plt.imshow(original, cmap=plt.cm.gray)
plt.title("Original image")

plt.subplot(1, 3, 2)
noisy_t = addingNoise(original, 0, 10)
plt.imshow(noisy_t, cmap=plt.cm.gray)
plt.title("Noisy image - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(noisy_t, original), 4)))

plt.subplot(1, 3, 3)
plt.imshow(denoised_images[15], cmap=plt.cm.gray)
plt.title("BayesShrink - Soft thresholding - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(denoised_images[15], original), 4)))

plt.show()


Fixed $ \sigma = 10 $ for the amount of noise in the image, we will report the best denoised images obtained by the two thresholding method. 

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(2, 2, 1)
plt.imshow(original, cmap=plt.cm.gray)
plt.title("Original image")

plt.subplot(2, 2, 2)
noisy_t = addingNoise(original, 0, 10)
plt.imshow(noisy_t, cmap=plt.cm.gray)
plt.title("Noisy image - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(noisy_t, original), 4)))

plt.subplot(2, 2, 3)
plt.imshow(denoised_images[16], cmap=plt.cm.gray)
plt.title("VisuShrinkMod - Hard thresholding - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(denoised_images[16], original), 4)))

plt.subplot(2, 2, 4)
plt.imshow(denoised_images[15], cmap=plt.cm.gray)
plt.title("BayesShrink - Soft thresholding - sigma: 10 - PSNR: {}".format(
    np.round(PSNR(denoised_images[15], original), 4)))

plt.show()


What we can say is that the best possible combination of parameters is soft thresholding with *BayesShrink* which returns a denoised image that is less blurred and with the intensity of the color closer to that of the original image than the one obtained using hard thresholding with *VisuShrinkMod*. A confirmation of this observation is once again given by the higher value recorded for the PSNR, the fidelity parameter.
<br><br>

---

## 5. Conclusions and considerations

Haar wavelets can be a very useful and powerful tool to clear the images from the noise, particularly Gaussian noise, which represents a very good candidate to replace other denoising methods.

We have reported how the hard and soft thresholding work and how much difficult the choice of the threshold parameter can be.

So, we introduce some "standard method" to define a value for this parameter which we have found in different works of other people.

By the results we obtained, we have seen that the best possible configuration of the denoising procedure with Haar wavelets is using the soft thresholding method to operate over the details coefficients and to consider the *BayesShrink* method to define the value of the threshold.

The denoising images have a good denoising quality which tends to decrease as the noise increases, in particular the standard deviation of the Gaussian noise: in fact, with higher level of noise, the resulting images can appear more grainy and blurry.

<br>

---

<br>

Finally, I can say that is a very good and simpler solution that can be useful in many cases and I had fun to implement it and to discover some "tricks" that helped me in solving the tasks.

It was a job that took a long time but that led me to fully understand the concepts, its usefulness and that thrilled me.

<br>

---

## 6. References

A list of all the articles, works and projects used to understand DBSCAN and build this laboratory:

1.   Yang Qiang, "Image denoising based on Haar wavelet transform".
     <br>*Proceedings of 2011 International Conference on Electronics and Optoelectronics, 2011, pp. V3-129-V3-132, doi: 10.1109/ICEOE.2011.6013318.*

2.   J. Pang, "Improved image denoising based on Haar wavelet transform".
     <br>*2017 IEEE SmartWorld, Ubiquitous Intelligence & Computing, Advanced & Trusted Computed, Scalable Computing & Communications, Cloud & Big Data Computing, Internet of People and Smart City Innovation (SmartWorld/SCALCOM/UIC/ATC/CBDCom/IOP/SCI), 2017, pp. 1-6, doi: 10.1109/UIC-ATC.2017.8397456.*

3.   S. H. Ismael, F. M. Mustafa and I. T. Okümüs, "A New Approach of Image Denoising Based on Discrete Wavelet Transform".
     <br>*2016 World Symposium on Computer Applications & Research (WSCAR), 2016, pp. 36-40, doi: 10.1109/WSCAR.2016.30.*

4.   Shivani Mupparaju, B Naga Venkata Satya Durga Jahnavi, "Comparison of Various Thresholding Techniques of Image Denoising".
     <br>*Department of ECE, VNR VJIET, Hyderabad, A.P, India - International Journal of Engineering Research & Technology (IJERT), Vol. 2 Issue 9, September - 2013, ISSN: 2278-0181*.

5.   Gregory R. Lee, Ralf Gommers, Filip Wasilewski, Kai Wohlfahrt, Aaron O’Leary (2019). "PyWavelets: A Python package for wavelet analysis". 
     <br>*Journal of Open Source Software, 4(36), 1237, https://doi.org/10.21105/joss.01237.*
